# 🇰🇭 Khmer OCR Seq2Seq Training on Google Colab

**Attention-Based Khmer OCR Engine with On-the-Fly Training**

This notebook implements the complete training pipeline for the Khmer OCR system using:
- **On-the-fly image generation** for efficient training
- **Fixed validation set** (6,400 images) for reproducible evaluation
- **Resumable training** with Google Drive checkpoint saving
- **GPU optimization** for Google Colab Pro/Pro+

**Target Performance:** ≤ 1.0% Character Error Rate (CER)

---

## 📋 Prerequisites

**Required:**
- Google Colab Pro/Pro+ (recommended for GPU memory)
- Google Drive account (for checkpoint storage)
- GitHub access to clone the repository

**Recommended Runtime:**
- GPU: Tesla V100, A100, or T4 (minimum 16GB VRAM)
- RAM: High-RAM runtime (25GB+)
- Storage: 50GB+ Google Drive space for checkpoints

---

In [1]:
# Check GPU availability and specs
import torch
import subprocess
import psutil
import os

print("🔧 System Information:")
print(f"   Python version: {torch.version}")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"   CUDA version: {torch.version.cuda}")
else:
    print("   ⚠️  No GPU available - switch to GPU runtime!")

print(f"   RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
print(f"   CPU cores: {psutil.cpu_count()}")

# Check available disk space
disk_usage = psutil.disk_usage('/')
print(f"   Disk space: {disk_usage.free / 1024**3:.1f} GB free / {disk_usage.total / 1024**3:.1f} GB total")

🔧 System Information:
   Python version: <module 'torch.version' from 'c:\\Users\\kunth\\miniforge3\\envs\\ml311\\Lib\\site-packages\\torch\\version.py'>
   PyTorch version: 2.7.1+cpu
   CUDA available: False
   ⚠️  No GPU available - switch to GPU runtime!
   RAM: 31.9 GB
   CPU cores: 8
   Disk space: 10.7 GB free / 3726.0 GB total


In [2]:
# Mount Google Drive for checkpoint storage
from google.colab import drive
import os

print("📁 Mounting Google Drive...")
drive.mount('/content/drive')

# Create checkpoint directory structure
checkpoint_base = '/content/drive/MyDrive/KhmerOCR_Checkpoints'
os.makedirs(checkpoint_base, exist_ok=True)
os.makedirs(f'{checkpoint_base}/models', exist_ok=True)
os.makedirs(f'{checkpoint_base}/logs', exist_ok=True)
os.makedirs(f'{checkpoint_base}/validation_set', exist_ok=True)

print(f"✅ Google Drive mounted at: {checkpoint_base}")

# List existing checkpoints
model_dir = f'{checkpoint_base}/models'
if os.path.exists(model_dir):
    checkpoints = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    if checkpoints:
        print(f"📦 Found {len(checkpoints)} existing checkpoints:")
        for cp in sorted(checkpoints):
            print(f"   - {cp}")
    else:
        print("📦 No existing checkpoints found")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Clone the Khmer OCR repository
import subprocess
import os

repo_url = "https://github.com/kunthet/khmer-ocr-seq2seq.git"
repo_dir = "/content/khmer-ocr-seq2seq"

# Remove existing directory if it exists
if os.path.exists(repo_dir):
    print("🗑️  Removing existing repository...")
    subprocess.run(["rm", "-rf", repo_dir], check=True)

# Clone the repository
print(f"📥 Cloning repository from {repo_url}...")
result = subprocess.run(["git", "clone", repo_url, repo_dir], 
                       capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Repository cloned successfully!")
    
    # Change to repository directory
    os.chdir(repo_dir)
    print(f"📂 Changed to directory: {os.getcwd()}")
    
    # Show recent commits
    commits = subprocess.run(["git", "log", "--oneline", "-5"], 
                           capture_output=True, text=True)
    print("\n📜 Recent commits:")
    print(commits.stdout)
else:
    print(f"❌ Failed to clone repository: {result.stderr}")
    raise Exception("Repository clone failed")

In [ ]:
# Install Python dependencies
print("📦 Installing Python dependencies...")

# Install packages from requirements.txt if it exists
if os.path.exists('requirements.txt'):
    !pip install -r requirements.txt
else:
    # Install essential packages
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install pillow numpy matplotlib tqdm
    !pip install tensorboard wandb
    !pip install opencv-python-headless
    !pip install scikit-learn
    !pip install Pillow-SIMD  # For better image processing performance

# Install additional Colab-specific packages
!pip install ipywidgets
!pip install plotly

print("✅ Dependencies installed successfully!")

# Verify installation
import torch
import torchvision
import PIL
import cv2
import numpy as np
import matplotlib.pyplot as plt

print(f"\n🔍 Package versions:")
print(f"   PyTorch: {torch.__version__}")
print(f"   Torchvision: {torchvision.__version__}")
print(f"   Pillow: {PIL.__version__}")
print(f"   OpenCV: {cv2.__version__}")
print(f"   NumPy: {np.__version__}")

In [ ]:
# Generate or load fixed validation set
validation_local = "data/validation_fixed"
validation_drive = f"{checkpoint_base}/validation_set"

print("🔍 Checking validation set...")

# Check if validation set exists on Google Drive
if os.path.exists(f"{validation_drive}/metadata.json"):
    print("✅ Validation set found on Google Drive")
    
    # Copy from Google Drive to local
    print("📥 Copying validation set from Google Drive...")
    !cp -r "{validation_drive}" "{validation_local}"
    
    # Verify local copy
    if os.path.exists(f"{validation_local}/metadata.json"):
        print("✅ Validation set copied successfully!")
    else:
        print("❌ Failed to copy validation set")
        
elif os.path.exists(f"{validation_local}/metadata.json"):
    print("✅ Validation set found locally")
    
    # Copy to Google Drive for backup
    print("📤 Backing up validation set to Google Drive...")
    !cp -r "{validation_local}" "{validation_drive}"
    print("✅ Validation set backed up to Google Drive")
    
else:
    print("❌ No validation set found. Generating new one...")
    
    # Generate validation set
    corpus_dir = "data/processed"
    fonts_dir = "fonts"
    
    if os.path.exists(corpus_dir) and os.path.exists(fonts_dir):
        print("🔄 Generating 6,400 validation images...")
        
        # Run validation set generation
        !python generate_fixed_validation_set.py --num-samples 6400 --output-dir "{validation_local}" --corpus-dir "{corpus_dir}" --fonts-dir "{fonts_dir}"
        
        # Check if generation was successful
        if os.path.exists(f"{validation_local}/metadata.json"):
            print("✅ Validation set generated successfully!")
            
            # Backup to Google Drive
            print("📤 Backing up to Google Drive...")
            !cp -r "{validation_local}" "{validation_drive}"
            print("✅ Validation set backed up to Google Drive")
        else:
            print("❌ Failed to generate validation set")
    else:
        print("❌ Cannot generate validation set - missing corpus or fonts")

# Verify validation set
if os.path.exists(f"{validation_local}/metadata.json"):
    import json
    with open(f"{validation_local}/metadata.json", 'r') as f:
        metadata = json.load(f)
    
    print(f"\n📊 Validation Set Statistics:")
    print(f"   Total samples: {metadata['total_samples']:,}")
    print(f"   Fonts used: {len(metadata['fonts_used'])}")
    print(f"   Image height: {metadata['generation_config']['image_height']}px")
    print(f"   Variable width: {metadata['generation_config']['variable_width']}")
    print(f"   Random seed: {metadata['generation_config']['random_seed']}")

In [ ]:
# Training configuration
TRAINING_CONFIG = {
    # Model parameters
    "vocab_size": 117,  # As per PRD
    "embedding_dim": 256,
    "encoder_hidden_size": 256,
    "decoder_hidden_size": 512,
    "attention_dim": 256,
    "image_height": 32,
    "image_channels": 1,
    
    # Training parameters
    "batch_size": 32,  # Will be adjusted based on GPU memory
    "learning_rate": 1e-6,  # As per PRD
    "num_epochs": 150,  # As per PRD
    "teacher_forcing_ratio": 1.0,  # As per PRD
    
    # On-the-fly generation parameters
    "samples_per_epoch": 10000,  # Number of samples to generate per epoch
    "max_sequence_length": 256,
    
    # Checkpoint and logging
    "save_every_n_epochs": 5,
    "validate_every_n_epochs": 1,
    "log_every_n_batches": 100,
    
    # Paths
    "corpus_dir": "data/processed",
    "fonts_dir": "fonts",
    "validation_dir": "data/validation_fixed",
    "checkpoint_dir": f"{checkpoint_base}/models",
    "log_dir": f"{checkpoint_base}/logs",
    
    # Performance
    "num_workers": 2,  # For DataLoader
    "pin_memory": True,
    "mixed_precision": True,  # Use automatic mixed precision
}

# Auto-adjust batch size based on GPU memory
if torch.cuda.is_available():
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    if gpu_memory_gb >= 40:  # A100 40GB
        TRAINING_CONFIG["batch_size"] = 64
    elif gpu_memory_gb >= 24:  # RTX 3090, RTX 4090
        TRAINING_CONFIG["batch_size"] = 48
    elif gpu_memory_gb >= 16:  # V100, P100
        TRAINING_CONFIG["batch_size"] = 32
    elif gpu_memory_gb >= 12:  # RTX 3060, RTX 4060
        TRAINING_CONFIG["batch_size"] = 24
    else:  # T4, K80
        TRAINING_CONFIG["batch_size"] = 16
    
    print(f"🎯 Auto-adjusted batch size to {TRAINING_CONFIG['batch_size']} for {gpu_memory_gb:.1f}GB GPU")

# Print configuration
print("\n⚙️  Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    if not key.endswith('_dir'):
        print(f"   {key}: {value}")

print("\n📁 Paths:")
for key, value in TRAINING_CONFIG.items():
    if key.endswith('_dir'):
        print(f"   {key}: {value}")

In [ ]:
# Run the on-the-fly training
print("🚀 Starting Khmer OCR Training...")

# Use the new on-the-fly training script
!python src/training/train_onthefly.py \
    --corpus-dir "{TRAINING_CONFIG['corpus_dir']}" \
    --fonts-dir "{TRAINING_CONFIG['fonts_dir']}" \
    --validation-dir "{TRAINING_CONFIG['validation_dir']}" \
    --checkpoint-dir "{TRAINING_CONFIG['checkpoint_dir']}" \
    --batch-size {TRAINING_CONFIG['batch_size']} \
    --learning-rate {TRAINING_CONFIG['learning_rate']} \
    --num-epochs {TRAINING_CONFIG['num_epochs']} \
    --samples-per-epoch {TRAINING_CONFIG['samples_per_epoch']} \
    --save-every {TRAINING_CONFIG['save_every_n_epochs']} \
    --validate-every {TRAINING_CONFIG['validate_every_n_epochs']} \
    --mixed-precision \
    --resume

print("✅ Training completed!")

In [ ]:
# Visualize training results
import matplotlib.pyplot as plt
import json
import os

# Load training history
history_path = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'training_history.json')

if os.path.exists(history_path):
    with open(history_path, 'r') as f:
        training_history = json.load(f)
    
    # Extract data for plotting
    epochs = [h['epoch'] for h in training_history]
    train_losses = [h['train_loss'] for h in training_history]
    val_losses = [h['val_loss'] for h in training_history if h['val_loss'] > 0]
    val_cers = [h['val_cer'] for h in training_history if h['val_cer'] > 0]
    
    val_epochs = [h['epoch'] for h in training_history if h['val_cer'] > 0]
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Khmer OCR Training Results', fontsize=16)
    
    # Training loss
    axes[0, 0].plot(epochs, train_losses, 'b-', label='Training Loss')
    if val_losses:
        axes[0, 0].plot(val_epochs, val_losses, 'r-', label='Validation Loss')
    axes[0, 0].set_title('Training & Validation Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Character Error Rate
    if val_cers:
        axes[0, 1].plot(val_epochs, val_cers, 'g-', label='CER', linewidth=2)
        axes[0, 1].axhline(y=0.01, color='r', linestyle='--', label='Target 1% CER')
        axes[0, 1].set_title('Character Error Rate (CER)')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('CER')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
    
    # Learning curve (CER vs Epochs)
    if val_cers:
        axes[1, 0].semilogy(val_epochs, val_cers, 'orange', label='CER (log scale)')
        axes[1, 0].axhline(y=0.01, color='r', linestyle='--', label='Target 1%')
        axes[1, 0].set_title('Learning Curve (CER Log Scale)')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('CER (log scale)')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Performance summary
    axes[1, 1].axis('off')
    if val_cers:
        best_cer = min(val_cers)
        final_cer = val_cers[-1]
        target_achieved = best_cer <= 0.01
        
        summary_text = f"""
        Training Summary:
        
        Total Epochs: {len(epochs)}
        Final Training Loss: {train_losses[-1]:.4f}
        Final Validation CER: {final_cer:.4f}
        Best Validation CER: {best_cer:.4f}
        
        Target Achievement:
        Target CER ≤ 1%: {'✅ ACHIEVED' if target_achieved else '❌ Not achieved'}
        
        Model Performance:
        {'🏆 Excellent!' if best_cer <= 0.005 else '🎯 Good!' if best_cer <= 0.01 else '📈 Needs improvement'}
        """
        
        axes[1, 1].text(0.1, 0.9, summary_text, transform=axes[1, 1].transAxes, 
                        fontsize=12, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    
    # Save plot to Google Drive
    plot_path = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'training_results.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Training results plot saved: {plot_path}")
    
else:
    print("❌ No training history found")

In [ ]:
# Final training summary
print("🎯 KHMER OCR TRAINING SUMMARY")
print("="*50)

# Load and display final results
best_model_path = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'best_model.pth')
latest_checkpoint = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'latest_checkpoint.pth')

if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path, map_location='cpu')
    
    print(f"📊 Best Model Performance:")
    print(f"   Epoch: {checkpoint['epoch']}")
    print(f"   Validation CER: {checkpoint['val_cer']:.4f}")
    print(f"   Training Loss: {checkpoint['train_loss']:.4f}")
    print(f"   Validation Loss: {checkpoint['val_loss']:.4f}")
    
    # Target achievement
    target_achieved = checkpoint['val_cer'] <= 0.01
    print(f"\n🎯 Target Achievement:")
    print(f"   Target CER ≤ 1%: {'✅ ACHIEVED' if target_achieved else '❌ Not achieved'}")
    
    if target_achieved:
        print(f"   🏆 Congratulations! Your model achieved the target performance!")
    else:
        print(f"   📈 Consider continuing training or adjusting hyperparameters")
        
elif os.path.exists(latest_checkpoint):
    checkpoint = torch.load(latest_checkpoint, map_location='cpu')
    print(f"📊 Latest Model Performance:")
    print(f"   Epoch: {checkpoint['epoch']}")
    print(f"   Validation CER: {checkpoint['val_cer']:.4f}")
    print(f"   Training Loss: {checkpoint['train_loss']:.4f}")
    
else:
    print("❌ No model checkpoints found")

print(f"\n💾 Checkpoints Location:")
print(f"   Google Drive: {TRAINING_CONFIG['checkpoint_dir']}")
print(f"   Local: /content/khmer-ocr-seq2seq/")

print(f"\n🔗 Resources:")
print(f"   Repository: https://github.com/kunthet/khmer-ocr-seq2seq.git")
print(f"   Documentation: docs/ON_THE_FLY_TRAINING.md")
print(f"   PRD: docs/prd.md")

print("\n" + "="*50)
print("🎉 Training session completed successfully!")
print("💡 All checkpoints and models are saved to Google Drive")
print("🚀 Your Khmer OCR model is ready for deployment!")

# Instructions for next steps
print("\n📋 Next Steps:")
print("   1. Download the best model from Google Drive")
print("   2. Test the model on new Khmer text images")
print("   3. Deploy the model in your application")
print("   4. Monitor performance and fine-tune if needed")